In [35]:
from match_functions import get_match_database
data = get_match_database('2025-03-01', '2025-03-15', leagues = ["Ligue 1"], save = True, add = False)

Extraction des liens...



100%|██████████| 15/15 [02:05<00:00,  8.38s/it]


Chargement de la base de données...
 
Extraction terminée en  4.52 minutes


In [38]:
import ace_tools_open as tools
tools.display_dataframe_to_user("Data", data)

Data


In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime

def get_page_rate(url):
    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)
    
    try:
        driver.get(url)

        # Attendre que les stats des joueurs apparaissent ET contiennent du texte
        WebDriverWait(driver, 15).until(
            lambda d: any(e.text.strip() for e in d.find_elements(By.CLASS_NAME, "player-stat-value"))
        )

        html = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    return soup

def get_rate(url):
    soup = get_page_rate(url)

    # Extraction des notes
    rate = [c.text.strip() if c.text.strip() else "NaN" for c in soup.find_all("span", class_="player-stat-value")]

    # Extraction des noms des joueurs
    player = [c.text.strip() for c in soup.find_all("span", class_="player-name")][:len(rate)]
    
    # Team
    code = soup.find_all("a", class_="team-link")
    team1 = [code[0].text] * len(player)
    team2 = [code[1].text] * len(player)

    # Date
    c = soup.find_all("div", class_="info-block cleared")[2].text.replace("Coup d'envoi:", "").replace("Date:Sun, ", "/").split("/")
    date_str = c[1]
    date_obj = datetime.strptime(date_str, '%d-%b-%y')
    formatted_date = date_obj.strftime('%d/%m/%Y')
    date = [formatted_date] * len(player)
    heure = [c[0]] * len(player)

    # Création du DataFrame
    df_rate = pd.DataFrame({'Date': date, 'Heure': heure, 'Equipe1': team1, 'Equipe2': team2, 'Player': player, 'Rate': rate})

    return df_rate


In [6]:
# 🔥 Test
url = "https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg"
df = get_rate(url)
import ace_tools_open as tools
tools.display_dataframe_to_user("", df)

In [16]:
soup = get_page_rate(url)

⚠️ Pas de cookies à accepter


In [67]:
def get_affluence(soup):
    code = soup.find_all("a", class_="team-link")
    team = [code[0].text, code[1].text]

    c = soup.find_all("div", class_="info-block cleared")[2].text.replace("Coup d'envoi:", "").replace("Date:Sun, ", "/").split("/")
    date_str = c[1]
    date_obj = datetime.strptime(date_str, '%d-%b-%y')  # Convertir en objet datetime
    formatted_date = date_obj.strftime('%d/%m/%Y')  # Reformater en "01/03/2025"
    date = [formatted_date, c[0]]

    affluence = [int(soup.find_all("span", class_="value")[1].text.replace(",", ""))]

    return date + team + affluence

In [68]:
get_affluence(soup)

['02/03/2025', '17:15', 'Auxerre', 'Strasbourg', 16495]

In [17]:
url = "https://fr.whoscored.com/regions/74/tournaments/22/seasons/10329/stages/23414/fixtures/france-ligue-1-2024-2025"
options = Options()
options.add_argument("--headless")  # Mode headless activé
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)

try:
    driver.get(url)

    # Attendre que les stats des joueurs apparaissent ET contiennent du texte
    #WebDriverWait(driver, 15).until(
    #    lambda d: any(e.text.strip() for e in d.find_elements(By.CLASS_NAME, "player-stat-value"))
    #)

    html = driver.page_source
finally:
    driver.quit()

soup = BeautifulSoup(html, "html.parser")

In [50]:
links = []
code = soup.find_all("div", class_="Match-module_right_oddsOn__o-ux-")
for c in code:
    if c.find("a") != None:
        links.append("https://fr.whoscored.com"+c.find("a").get("href"))
links

['https://fr.whoscored.com/matches/1824845/live/france-ligue-1-2024-2025-saint-etienne-nice',
 'https://fr.whoscored.com/matches/1824846/live/france-ligue-1-2024-2025-lens-le-havre',
 'https://fr.whoscored.com/matches/1824841/live/france-ligue-1-2024-2025-paris-saint-germain-lille',
 'https://fr.whoscored.com/matches/1824843/live/france-ligue-1-2024-2025-lyon-brest',
 'https://fr.whoscored.com/matches/1824844/live/france-ligue-1-2024-2025-montpellier-rennes',
 'https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg',
 'https://fr.whoscored.com/matches/1824849/live/france-ligue-1-2024-2025-angers-toulouse',
 'https://fr.whoscored.com/matches/1824842/live/france-ligue-1-2024-2025-marseille-nantes',
 'https://fr.whoscored.com/matches/1824853/live/france-ligue-1-2024-2025-toulouse-monaco',
 'https://fr.whoscored.com/matches/1824857/live/france-ligue-1-2024-2025-rennes-paris-saint-germain',
 'https://fr.whoscored.com/matches/1824856/live/france-ligue-1-202

In [ ]:
<button id="dayChangeBtn-prev" class="Calendar-module_dayChangeBtn__sEvC8"><div class="ThemeImage-module_themeImage__mmLe2"><img src="https://cdn.whoscored.com/theme/svg/chevronLeft-43e59a635e.svg" height="10" width="8" alt="chevron-left"></div></button>

In [53]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import time

# URL de base
url = "https://fr.whoscored.com/regions/74/tournaments/22/seasons/10329/stages/23414/fixtures/france-ligue-1-2024-2025"

# Configuration de Selenium
options = Options()
options.add_argument("--headless")  # Mode headless activé
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)

all_links = []

try:
    driver.get(url)
    
    # Accepter les cookies si la bannière apparaît
    try:
        accept_cookies = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "qc-cmp-cleanslate"))
        )
        driver.execute_script("arguments[0].click();", accept_cookies)
        time.sleep(2)
    except Exception:
        print("Bannière de cookies non détectée.")
    
    for _ in range(3):  # Boucle pour trois mois
        time.sleep(2)  # Attendre le chargement de la page
        
        # Récupération du HTML
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        # Extraction des liens
        code = soup.find_all("div", class_="Match-module_right_oddsOn__o-ux-")
        for c in code:
            if c.find("a") is not None:
                all_links.append("https://fr.whoscored.com" + c.find("a").get("href"))
        
        # Faire défiler la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Cliquer sur le bouton pour changer de mois
        try:
            prev_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "dayChangeBtn-prev"))
            )
            driver.execute_script("arguments[0].click();", prev_button)  # Utilisation de JavaScript
        except Exception as e:
            print("Impossible de cliquer sur le bouton du mois précédent :", e)
            break
        
finally:
    driver.quit()

print(all_links)


['https://fr.whoscored.com/matches/1824845/live/france-ligue-1-2024-2025-saint-etienne-nice', 'https://fr.whoscored.com/matches/1824846/live/france-ligue-1-2024-2025-lens-le-havre', 'https://fr.whoscored.com/matches/1824841/live/france-ligue-1-2024-2025-paris-saint-germain-lille', 'https://fr.whoscored.com/matches/1824843/live/france-ligue-1-2024-2025-lyon-brest', 'https://fr.whoscored.com/matches/1824844/live/france-ligue-1-2024-2025-montpellier-rennes', 'https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg', 'https://fr.whoscored.com/matches/1824849/live/france-ligue-1-2024-2025-angers-toulouse', 'https://fr.whoscored.com/matches/1824842/live/france-ligue-1-2024-2025-marseille-nantes', 'https://fr.whoscored.com/matches/1824853/live/france-ligue-1-2024-2025-toulouse-monaco', 'https://fr.whoscored.com/matches/1824857/live/france-ligue-1-2024-2025-rennes-paris-saint-germain', 'https://fr.whoscored.com/matches/1824856/live/france-ligue-1-2024-2025-lil

In [54]:
all_links

['https://fr.whoscored.com/matches/1824845/live/france-ligue-1-2024-2025-saint-etienne-nice',
 'https://fr.whoscored.com/matches/1824846/live/france-ligue-1-2024-2025-lens-le-havre',
 'https://fr.whoscored.com/matches/1824841/live/france-ligue-1-2024-2025-paris-saint-germain-lille',
 'https://fr.whoscored.com/matches/1824843/live/france-ligue-1-2024-2025-lyon-brest',
 'https://fr.whoscored.com/matches/1824844/live/france-ligue-1-2024-2025-montpellier-rennes',
 'https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg',
 'https://fr.whoscored.com/matches/1824849/live/france-ligue-1-2024-2025-angers-toulouse',
 'https://fr.whoscored.com/matches/1824842/live/france-ligue-1-2024-2025-marseille-nantes',
 'https://fr.whoscored.com/matches/1824853/live/france-ligue-1-2024-2025-toulouse-monaco',
 'https://fr.whoscored.com/matches/1824857/live/france-ligue-1-2024-2025-rennes-paris-saint-germain',
 'https://fr.whoscored.com/matches/1824856/live/france-ligue-1-202

# Ttttttesttttt